In [5]:
!pip install qiskit qiskit_aer 

  Using cached qiskit_aer-0.17.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (8.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 3.4 MB/s eta 0:00:00 MB/s eta 0:00:01:01


In [2]:
!pip install tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)



[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import numpy as np
import pandas as pd
import os
import pickle
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from collections import Counter
from tqdm import tqdm

def amplitude_encode(features):
    d = len(features)
    n_qubits = int(np.ceil(np.log2(d)))
    dim = 2**n_qubits
    vec = np.zeros(dim)
    vec[:d] = features
    norm = np.linalg.norm(vec)
    return vec / norm if norm != 0 else vec

def build_state_prep(vec, n_qubits):
    qc = QuantumCircuit(n_qubits)
    qc.initialize(vec, range(n_qubits))
    return qc

def swap_test(vec1, vec2, shots=1024):
    n_qubits = int(np.log2(len(vec1)))
    qc = QuantumCircuit(1 + 2 * n_qubits, 1)
    qc.h(0)
    qc.compose(build_state_prep(vec1, n_qubits), qubits=range(1, 1 + n_qubits), inplace=True)
    qc.compose(build_state_prep(vec2, n_qubits), qubits=range(1 + n_qubits, 1 + 2 * n_qubits), inplace=True)
    for i in range(n_qubits):
        qc.cswap(0, i + 1, i + 1 + n_qubits)
    qc.h(0)
    qc.measure(0, 0)

    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    result = simulator.run(tqc, shots=shots).result()
    counts = result.get_counts()
    prob_0 = counts.get('0', 0) / shots
    fidelity = 2 * prob_0 - 1
    return fidelity

# Load dataset
df = pd.read_csv('data.csv')

# Model filename
model_file = 'quantum_knn_model1.pkl'

# Columns for features
feature_cols = ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']

# Initialize test data containers
X_test = []
y_test = []

if os.path.exists(model_file):
    print("Loading saved model...")
    with open(model_file, 'rb') as f:
        model_data = pickle.load(f)
        train_states = model_data['train_states']
        train_labels = model_data['train_labels']
        scaler = model_data['scaler']
        label_encoder = model_data['label_encoder']
        class_names = model_data['class_names']

    labels = df['label'].unique()
    for label in labels:
        label_data = df[df['label'] == label]
        X_test.append(label_data[feature_cols].iloc[15:20].values)
        y_test.append(label_data['label'].iloc[15:20].values)

else:
    print("Training model...")

    X_train, y_train = [], []
    for label in df['label'].unique():
        label_data = df[df['label'] == label]
        X_train.append(label_data[feature_cols].iloc[:15].values)
        y_train.extend(label_data['label'].iloc[:15].values)
        X_test.append(label_data[feature_cols].iloc[15:20].values)
        y_test.extend(label_data['label'].iloc[15:20].values)

    # Combine training data
    X_train = np.concatenate(X_train, axis=0)
    X_test = np.concatenate(X_test, axis=0)

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train)

    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_train)
    class_names = label_encoder.classes_.tolist()

    # Quantum amplitude encoding
    train_states = [amplitude_encode(xi) for xi in X_scaled]
    train_labels = y_encoded

    # Save model
    print("Saving trained model...")
    with open(model_file, 'wb') as f:
        pickle.dump({
            'train_states': train_states,
            'train_labels': train_labels,
            'scaler': scaler,
            'label_encoder': label_encoder,
            'class_names': class_names
        }, f)

# Prepare test data
X_test_scaled = scaler.transform(X_test)
y_test_encoded = label_encoder.transform(y_test)  # For accuracy calculation

# Predict
k = 5
predictions = []

print("\nRunning predictions...")
for xi in tqdm(X_test_scaled, desc="Predicting"):
    vec_test = amplitude_encode(xi)
    fidelities = [swap_test(vec_test, vec_train, shots=1024) for vec_train in train_states]
    distances = [1 - f for f in fidelities]
    nn_idx = np.argsort(distances)[:k]
    nearest = [train_labels[i] for i in nn_idx]
    predictions.append(Counter(nearest).most_common(1)[0][0])

# Evaluate
acc = accuracy_score(y_test_encoded, predictions)
print(f"\nAccuracy: {acc:.2f}")
print("\nClassification Report:\n")
print(classification_report(y_test_encoded, predictions, target_names=class_names))


Training model...
Saving trained model...

Running predictions...


Predicting: 100%|██████████| 110/110 [1:51:42<00:00, 60.93s/it]


Accuracy: 0.87

Classification Report:

              precision    recall  f1-score   support

       apple       1.00      1.00      1.00         5
      banana       1.00      1.00      1.00         5
   blackgram       0.80      0.80      0.80         5
    chickpea       1.00      1.00      1.00         5
     coconut       1.00      1.00      1.00         5
      coffee       1.00      1.00      1.00         5
      cotton       1.00      1.00      1.00         5
      grapes       0.83      1.00      0.91         5
        jute       0.75      0.60      0.67         5
 kidneybeans       0.62      1.00      0.77         5
      lentil       0.75      0.60      0.67         5
       maize       1.00      0.60      0.75         5
       mango       1.00      1.00      1.00         5
   mothbeans       0.67      0.80      0.73         5
    mungbean       1.00      1.00      1.00         5
   muskmelon       1.00      1.00      1.00         5
      orange       0.80      0.80      0

In [2]:
import pandas as pd

def extract_unique_labels(csv_file_path):
    # Load CSV file
    df = pd.read_csv(csv_file_path)

    # Check if 'label' column exists
    if 'label' not in df.columns:
        raise ValueError("The CSV file must contain a 'label' column.")

    # Get unique labels
    unique_labels = df['label'].unique()

    # Print them
    print("Distinct labels:")
    for label in unique_labels:
        print(label)

    return unique_labels

# Example usage
extract_unique_labels("data.csv")  # replace with your actual file path


Distinct labels:
rice
maize
chickpea
kidneybeans
pigeonpeas
mothbeans
mungbean
blackgram
lentil
pomegranate
banana
mango
grapes
watermelon
muskmelon
apple
orange
papaya
coconut
cotton
jute
coffee


array(['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas',
       'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate',
       'banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple',
       'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee'],
      dtype=object)

In [20]:
import pickle

with open('quantum_knn_model1.pkl', 'rb') as f:
    data = pickle.load(f)

print("Available keys:", data.keys())

Available keys: dict_keys(['train_states', 'train_labels', 'scaler', 'label_encoder', 'class_names'])
